In [85]:
from __future__ import division
import numpy as np
from math import *

In [86]:
# number of inputs
N = 100
# number of nodes in a input
D = 2
# number of nodes in hidden layer
M = 4
# number of nodes in output
K = 1
# learning rate
rate = 0.12
# hidden relationship between input and output 
operation = "XOR"
# number of back propagations
iterations = 100


# input generation
X = np.random.randint(2, size = (D,N))
X_bias = np.ones((1,N))

# labels 
Y = np.matrix(np.ones((N, K)))

for i in range (0,N):
    if operation == "XOR":
        Y[i,K - 1] = X.T[i,0] ^ X.T[i,1]
    if operation == "AND":
        Y[i,K - 1] = X.T[i,0] & X.T[i,1]
    if operation == "OR":
        Y[i,K - 1] = X.T[i,0] | X.T[i,1]
            

# adding noise to generate more labels
X = X.astype(np.float)

for i in range (0,D):
    for j in range (0,N):
        X[i][j] += np.random.normal(0, 1e-4)
        if i == 0:
            Y[j,K - 1] += np.random.normal(0, 1e-4)
            
X = np.matrix(np.concatenate((X_bias,X), axis = 0))

In [87]:
# initializing all parameters
alpha = np.matrix(np.zeros((D + 1, M))) 

# initialization of hidden nodes
Z = np.matrix(np.zeros((N,M+1)))

# initializing parameters
beta = np.matrix(np.zeros((M + 1, K)))

# initializing output
y_hat = np.matrix(np.zeros((N, K)))

# finds sigmoid
def sigmoid(val):
    return 1/(1 + exp(-val))

# finds Z(m,i) - m th hidden node value for i th input vector
def z_sigmoid(i, m, alpha):
    val = alpha.T[m]*((X.T[i]).T)
    return sigmoid(val)

# finds Y(k,i) - k th dimension value of the i th output
def y_sigmoid(i, k, beta, Z):
    val = beta.T[k] * (Z[i].T)
    return sigmoid(val)

# first derivative of sigmoid
def der_sigmoid(val):
    a = sigmoid(val)
    return np.exp(-val)*(a**2)

# functions to calculate intra math.
def delta(i, k, y_hat, beta, Z):
    return -2 * np.sum((Y.T[k,i] - y_hat[i,k])) * der_sigmoid(beta.T[k] * (Z[i].T))

def S_m(i, m, beta, y_hat, Z):
    s = 0
    for n in range (0,K):
        s += delta(i, n, y_hat, beta, Z) * np.sum(beta[m,n])
    return s

def R_alpha(i, m, beta, alpha, l, y_hat, Z):
    a = S_m(i, m, beta, y_hat, Z) * der_sigmoid(alpha.T[m]*((np.matrix(X.T[i])).T))
    return a*np.sum(X.T[i,l])

def sum_R_alpha(l, m, beta, alpha, y_hat, Z):
    s = 0
    for i in range (0,N):
        s = s + R_alpha(i, m, beta, alpha, l, y_hat, Z)
    return s

def sum_R_beta(m, k, y_hat, beta, Z):
    s = 0
    for i in range (0, N):
        s = s + delta(i, k, y_hat, beta, Z)*Z[i,m]
    return s

In [88]:
for n in range (0,iterations):
    
    for i in range (0,N):
        Z[i,0] = 1
        for j in range (1,M + 1):
            Z[i,j] = z_sigmoid(i, j - 1, alpha)

    for i in range (0, N):
        for j in range (0, K):
            y_hat[i,j] = y_sigmoid(i, j, beta, Z) 
            
    for i in range (0, D + 1):
        for j in range (0, M):
            alpha[i,j] = alpha[i,j] - rate*sum_R_alpha(i, j, beta, alpha, y_hat, Z)

    for i in range (0, M + 1):
        for j in range (0, K):
            beta[i,j] = beta[i,j] - rate*sum_R_beta(i, j, y_hat, beta, Z)
            
    error = 0
    for i in range (0,N):
        error += (Y[i,K - 1] - y_hat[i,K - 1])**2
    if n > 0 :
        print error

# applied back propagation method to find optimal parameters 
# error is printed for each iteration, so that performance can be known
# all parameters are updated simultaneously within a iteration
# code is configurable for - 
# 1.no. of inputs, 2.no of hidden nodes, 3. hidden relation among input and output (XOR, AND, OR), 4.no of iterations

25.985706962545255
28.460609966139383
30.431922835599636
29.310187140425164
29.86518210929177
26.98551190152489
27.52385252847392
26.253294531277522
26.54991890063546
25.643290219177743
25.796808838729284
25.191224853912864
25.252916517506616
24.8321085760247
24.83159242488469
24.52308798328338
24.47677024651292
24.234436474838482
24.150154661472726
23.942298005043405
23.8219119689476
23.624738152674283
23.46588343093021
23.26022645083836
23.05811378154371
22.82845748507835
22.578249712942036
22.31338449727953
22.013036657737477
21.70746608715643
21.36041728641692
21.015858637479376
20.63305125945915
20.259702607977424
19.860021602685176
19.476182503001706
19.08327139483031
18.710925310666063
18.345730090444235
18.00288434484721
17.677046388109247
17.373062760556557
17.08950318403018
16.826685458937362
16.584047564696952
16.36049338410586
16.15484299996057
15.965237485724254
15.789490075425526
15.624810178106937
15.467863944557752
15.314724805137791
15.160962221535986
15.001815302905
1